# Lost in Cleaning Data? 
There's so many approaches to cleaning data. It's easy to get lost. 

![](https://media.giphy.com/media/YR2QxyyVR6byo/giphy.gif)

Let me take you through a basic approach to **cleaning data**:
1. [Understand/View](#one)
2. [Select/Rename](#two)
3. [Create/Drop Columns](#three)
4. [Convert Data Types](#four)
5. [Handle Missing Values](#five)
6. [Scaling/Transforms](#six)

In [17]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [18]:
# Define constants
SELECT_COLS = ['Market/Regular ',  'actual_eta', 'Org_lat_lon', 'Des_lat_lon', 'ontime',
'delay','trip_start_date', 'trip_end_date', 'TRANSPORTATION_DISTANCE_IN_KM', 
 'Material Shipped']
DATE_COLS = ['trip_start_date', 'trip_end_date']
FLOAT_COLS = ['transportation_distance_in_km','origin_lat','origin_lon','des_lat','des_lon','days_taken']
CAT_COLS = ['market_regular','material_shipped']
REPLACE_REGEX_DICT = {r'(?!^)([A-Z]+)':r'_\1','/| ': '','__':'_','T_R':'TR','/':''}
DROP_COLS = ['ontime', 'delay','trip_start_date','trip_end_date','actual_eta','org_lat_lon','des_lat_lon']

<a id = 'one'></a>
## Understand/View 

For this example we'll use a simple dataset: [Delivery truck trips data](https://www.kaggle.com/ramakrishnanthiyagu/delivery-truck-trips-data).

Some features we'll look at (of course we could examine more):
- Market/Regular - Type of trip. Regular - Vendors with whom we will have contract. Market - Vendor with whom we will not have contract
- Orglatlon - Latitude/Longitude of start place
- Des lat lon - Latitude/Longitude of end place
- TRANSPORTATION_DISTANCE_IN_KM - Total KM of travel
- Material Shipped - Type of materials in delivery
- days_taken - calculated from difference between columns; `trip_end_date` and `trip_start_date`

![](https://media.giphy.com/media/xT9C25UNTwfZuk85WP/giphy.gif)


In [36]:
df = pd.read_csv('logistics-data.csv')
# df = pd.read_excel('logistics-data.xlsx')
df.info()
df[SELECT_COLS].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6880 entries, 0 to 6879
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   GpsProvider                         5927 non-null   object 
 1   BookingID                           6880 non-null   object 
 2   Market/Regular                      6880 non-null   object 
 3   BookingID_Date                      6880 non-null   object 
 4   vehicle_no                          6880 non-null   object 
 5   Origin_Location                     6880 non-null   object 
 6   Destination_Location                6880 non-null   object 
 7   Org_lat_lon                         6880 non-null   object 
 8   Des_lat_lon                         6880 non-null   object 
 9   Data_Ping_time                      5927 non-null   object 
 10  Planned_ETA                         6880 non-null   object 
 11  Current_Location                    5916 no

,Market/Regular,actual_eta,Org_lat_lon,Des_lat_lon,ontime,delay,trip_start_date,trip_end_date,TRANSPORTATION_DISTANCE_IN_KM,Material Shipped
0,Market,2020-08-28 14:38:04.447,"13.1550,80.1960","12.7400,77.8200",NaN,R,2020-08-17 14:59:01.000,NaN,320.0,BRACKET / GRAB HANDLE
1,Regular,2020-08-28 12:46:17.007,"12.8390,79.9540","12.8390,79.9540",G,NaN,2020-08-27 16:21:52.000,NaN,103.0,ZB MODEL PLATE / 3143
2,Regular,2020-08-28 16:03:30.793,"11.8710,79.7390","11.8710,79.7390",G,NaN,2020-08-27 17:57:04.000,NaN,300.0,LETTERING / FUSO
3,Regular,2020-08-28 12:50:27.997,"12.8390,79.9540","12.8390,79.9540",G,NaN,2020-08-28 00:47:45.000,NaN,61.0,LU STRUT RA / RADIUS ROD
4,Regular,2020-08-28 14:22:50.127,"11.8720,79.6320","11.8720,79.6320",G,NaN,2020-08-28 01:13:48.000,NaN,240.0,WISHBONE / V ROD/HDT


<a id = 'two'></a>
## Rename/Select

The column names are a bit of a mess. Better make them consistent with `snake_case` style (it's the Python standard, get it)!

![](https://media.giphy.com/media/3o6EhXODzc79cnZfRC/giphy.gif)


In [20]:
df = df[SELECT_COLS]
for col_name, rename_col in REPLACE_REGEX_DICT.items():
    df.columns = df.columns.str.replace(col_name, rename_col)
df.columns = df.columns.str.lower()
df.head()

<ipython-input-20-fd763e656130>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace(col_name, rename_col)


,market_regular,actual_eta,org_lat_lon,des_lat_lon,ontime,delay,trip_start_date,trip_end_date,transportation_distance_in_km,material_shipped
0,Market,2020-08-28 14:38:04.447,"13.1550,80.1960","12.7400,77.8200",NaN,R,2020-08-17 14:59:01.000,NaN,320.0,BRACKET / GRAB HANDLE
1,Regular,2020-08-28 12:46:17.007,"12.8390,79.9540","12.8390,79.9540",G,NaN,2020-08-27 16:21:52.000,NaN,103.0,ZB MODEL PLATE / 3143
2,Regular,2020-08-28 16:03:30.793,"11.8710,79.7390","11.8710,79.7390",G,NaN,2020-08-27 17:57:04.000,NaN,300.0,LETTERING / FUSO
3,Regular,2020-08-28 12:50:27.997,"12.8390,79.9540","12.8390,79.9540",G,NaN,2020-08-28 00:47:45.000,NaN,61.0,LU STRUT RA / RADIUS ROD
4,Regular,2020-08-28 14:22:50.127,"11.8720,79.6320","11.8720,79.6320",G,NaN,2020-08-28 01:13:48.000,NaN,240.0,WISHBONE / V ROD/HDT


<a id ='three'> </a>
## Create/Drop Columns
Data cleaning should be for a purpose; usually data visualization, analysis or modelling.

You create or drop data columns depending on what you've interested in looking at and what you're trying to achieve. 
### Create new columns: 
- `'origin_lat', 'origin_lon'`: Split latitude and longitude for place of origin into separate columns.
- `'des_lat', 'des_lon'`: Split latitude and longitude for destination into separate columns
- `'days_taken'`: Calculate `trip_end_date` - `trip_start_date`.  
### Drow rows/columns:
- Drop rows where `'days_taken'` is negative value (yeah, I'm not sure why?!)
- Drop columns that are unnecessary, given by `DROP_COLS`.

Drop it like a boss!

![](https://media.giphy.com/media/DfbpTbQ9TvSX6/giphy.gif)

In [21]:
df[['origin_lat', 'origin_lon']] = df['org_lat_lon'].str.split(',',expand=True) 
df[['des_lat', 'des_lon']] = df['des_lat_lon'].str.split(',',expand=True) 
df[DATE_COLS] = df[DATE_COLS].astype('datetime64')
days_taken = (df['trip_end_date']-df['trip_start_date'])/ np.timedelta64(1, 'D')
df.insert(0,'days_taken',days_taken)
df = df.drop(df[df['days_taken']<0].index, axis=0)
df = df.drop(DROP_COLS, axis=1)
df.head()

,days_taken,market_regular,transportation_distance_in_km,material_shipped,origin_lat,origin_lon,des_lat,des_lon
0,NaN,Market,320.0,BRACKET / GRAB HANDLE,13.1550,80.1960,12.7400,77.8200
1,NaN,Regular,103.0,ZB MODEL PLATE / 3143,12.8390,79.9540,12.8390,79.9540
2,NaN,Regular,300.0,LETTERING / FUSO,11.8710,79.7390,11.8710,79.7390
3,NaN,Regular,61.0,LU STRUT RA / RADIUS ROD,12.8390,79.9540,12.8390,79.9540
4,NaN,Regular,240.0,WISHBONE / V ROD/HDT,11.8720,79.6320,11.8720,79.6320


In [22]:
df.days_taken.value_counts()

0.097917     65
0.135417     54
0.093750     27
0.281250     25
0.331250     21
             ..
1.025428      1
4.968056      1
4.904861      1
19.796609     1
0.001000      1
Name: days_taken, Length: 5995, dtype: int64

<a id ='four'> </a>
## Convert Data Types
- `df.info()`: Easy to check all data types.
- `.astype()`: Convert data types easily; usually `float, int, category`.
 
![](https://media.giphy.com/media/kDmiZp6eXOgGunaAEe/giphy.gif)

In [23]:
df[FLOAT_COLS] = df[FLOAT_COLS].astype("float")
df[CAT_COLS] = df[CAT_COLS].astype("category")
df2 = df.copy()

<a id ='five'> </a>
## Handle Missing Values
- `df.isnull().sum()`: Easy to check the number of missing values in each column.
- We impute missing numeric values with the column mean. 

Since there are no missing values for categorical variables for this dataset, we commented out code for handling missing categorical values.

In [24]:
# Check for missing data
print(df.isnull().sum()) 
missing_values_df = df.loc[:, df.isnull().sum()>0]

# Handle missing numeric values only
numeric_features = missing_values_df.select_dtypes(include='float').columns
num_imputer = SimpleImputer(missing_values= np.nan, strategy='median')
df[numeric_features] = num_imputer.fit_transform(missing_values_df.select_dtypes(include='float'))

# handling missing categorical values (for your reference)
# categorical_features = missing_values_df.select_dtypes(include=['category']).columns
# cat_imputer = SimpleImputer(strategy='constant', fill_value=np.nan)
# df[categorical_features] = cat_imputer.fit_transform(missing_values_df)

print("\nImputing Missing Values:\n")
df.isnull().sum()

days_taken                       194
market_regular                     0
transportation_distance_in_km    707
material_shipped                   0
origin_lat                         0
origin_lon                         0
des_lat                            0
des_lon                            0
dtype: int64

Imputing Missing Values:



days_taken                       0
market_regular                   0
transportation_distance_in_km    0
material_shipped                 0
origin_lat                       0
origin_lon                       0
des_lat                          0
des_lon                          0
dtype: int64

<a id ='six'> </a>
## Scaling/Transforms
Scaling/transforms is useful for:
- Visualizing data on the same scale - less distortion or skewness.
- Data analysis techniques such as PCA, clustering or outlier detection. 
- Data modelling to prevent variables on a larger scale, dominating the model weights.

Basically, we want data columns on **similar scales**.

![](https://media.giphy.com/media/IcStLavfAdhoQ/giphy.gif)



In [25]:
y = df["days_taken"]
df.drop("days_taken",axis=1, inplace=True)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6812 entries, 0 to 6879
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   market_regular                 6812 non-null   category
 1   transportation_distance_in_km  6812 non-null   float64 
 2   material_shipped               6812 non-null   category
 3   origin_lat                     6812 non-null   float64 
 4   origin_lon                     6812 non-null   float64 
 5   des_lat                        6812 non-null   float64 
 6   des_lon                        6812 non-null   float64 
dtypes: category(2), float64(5)
memory usage: 382.6 KB


In [27]:
numeric_features = df.select_dtypes(exclude=['category']).columns
scaler = StandardScaler()
scaled_num_df = pd.DataFrame(scaler.fit_transform(df.loc[:,numeric_features]))
scaled_num_df.head()

,0,1,2,3,4
0,-0.260707,-0.848202,0.090764,-0.957792,-0.275004
1,-0.559058,-0.906743,0.037863,-0.941831,0.205213
2,-0.288205,-1.086073,-0.009135,-1.097896,0.156832
3,-0.616804,-0.906743,0.037863,-0.941831,0.205213
4,-0.370698,-1.085888,-0.032525,-1.097735,0.132753


Scaling numeric variables are easy but what about categorical variables?

Well you don't really scale categories but you may need to transform them especially for data models.

This is where **One Hot Encoding** comes in handy; convert categories to a matrix of zeros and ones. [More info here](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/).

In [28]:
categorical_features = df.select_dtypes(include=['category']).columns
encoder = OneHotEncoder(sparse=False)
df_cat_transformed = pd.DataFrame(encoder.fit_transform(df.loc[:, categorical_features]))
df_cat_transformed.head()

,0,1,2,3,4,5,6,7,8,9,...,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Join the scaled numeric data with the transformed categorical data.

What does your dataset look like now?


In [29]:
# First col is target variable y and the rest are features X
X_pre = pd.concat([scaled_num_df, df_cat_transformed], axis =1)
X_pre.head()

,0,1,2,3,4,0,1,2,3,4,...,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401
0,-0.260707,-0.848202,0.090764,-0.957792,-0.275004,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.559058,-0.906743,0.037863,-0.941831,0.205213,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.288205,-1.086073,-0.009135,-1.097896,0.156832,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.616804,-0.906743,0.037863,-0.941831,0.205213,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.370698,-1.085888,-0.032525,-1.097735,0.132753,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
pd.concat([y.reset_index(drop=True), pd.DataFrame(X_pre)], axis=1).head()

,days_taken,0,1,2,3,4,0,1,2,3,...,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401
0,3.624479,-0.260707,-0.848202,0.090764,-0.957792,-0.275004,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.624479,-0.559058,-0.906743,0.037863,-0.941831,0.205213,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.624479,-0.288205,-1.086073,-0.009135,-1.097896,0.156832,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.624479,-0.616804,-0.906743,0.037863,-0.941831,0.205213,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.624479,-0.370698,-1.085888,-0.032525,-1.097735,0.132753,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
# Or as a numpy array
y_pre = y.to_numpy().reshape(len(y), 1)
dataset = np.concatenate((y_pre, X_pre), axis=1)
dataset

array([[ 3.62447917, -0.26070704, -0.8482019 , ...,  0.        ,
         0.        ,  0.        ],
       [ 3.62447917, -0.55905847, -0.90674343, ...,  0.        ,
         0.        ,  0.        ],
       [ 3.62447917, -0.28820487, -1.0860732 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.17291667, -0.6333026 , -0.21736272, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04583333, -0.6333026 , -0.21736272, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.13541667, -0.6333026 , -0.21736272, ...,  0.        ,
         0.        ,  0.        ]])

# Alternate way: Shorter code

In [32]:
df2["days_taken"].to_numpy()

array([       nan,        nan,        nan, ..., 0.17291667, 0.04583333,
       0.13541667])

In [33]:
# Fill missing values for y data (do not scale as we want do not want to unscale for predictions)
num_imputer = SimpleImputer(missing_values= np.nan, strategy='median')
y_pre = num_imputer.fit_transform(df2["days_taken"].to_numpy().reshape(len(y), 1)) 
# y_pre = y_pre.flatten()
y_pre

array([[3.62447917],
       [3.62447917],
       [3.62447917],
       ...,
       [0.17291667],
       [0.04583333],
       [0.13541667]])

In [35]:
# Preprocess features X
numeric_features =  df.select_dtypes('number').columns
numeric_transformer = Pipeline(
steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)
categorical_features = df.select_dtypes(exclude='number').columns
categorical_transformer = Pipeline(
steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
]
)
preprocess = ColumnTransformer(
transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
]
)
X_pre = preprocess.fit_transform(df)
y_pre = y.to_numpy().reshape(len(y), 1)
dataset = np.concatenate((y_pre, X_pre.toarray()), axis=1)
dataset

array([[ 3.62447917, -0.26070704, -0.8482019 , ...,  0.        ,
         0.        ,  0.        ],
       [ 3.62447917, -0.55905847, -0.90674343, ...,  0.        ,
         0.        ,  0.        ],
       [ 3.62447917, -0.28820487, -1.0860732 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.17291667, -0.6333026 , -0.21736272, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04583333, -0.6333026 , -0.21736272, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.13541667, -0.6333026 , -0.21736272, ...,  0.        ,
         0.        ,  0.        ]])

Any questions? Let me know!

Otherwise, you're good to go 🙂